In [12]:
import sys
sys.path.append("../../")
from data.Frame import Frame
from bk import show_weighted_average
from bokeh.io import output_notebook
from bokeh.plotting import figure, output_file, show, reset_output
reset_output()
output_notebook()

Loading BokehJS ...

In [13]:
years = list(range(2001,2018))

In [3]:
show_weighted_average(years, 'NV.AGR.TOTL.ZS', 'NY.GDP.MKTP.CD', ['LDCs'])

In [ ]:
# Urban population as a percentage of total population
show_weighted_average('SP.RUR.TOTL.ZS', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs'])

In [ ]:
# Access to safely managed sanitation services
show_weighted_average('SH.STA.SMSS.ZS', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs'])

In [ ]:
# Literacy rate, youth female (% of females ages 15-24)
years = list(range(2011,2018))
show_weighted_average('SE.ADT.1524.LT.FE.ZS', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs'])

In [ ]:
# Literacy rate, youth male (% of male ages 15-24)
show_weighted_average('SE.ADT.1524.LT.MA.ZS', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs'])

In [ ]:
# School enrollment, tertiary
show_weighted_average('SE.TER.ENRR', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs', 'UMICs'])

In [ ]:
# Pupil teacher ration
years = list(range(2000,2018))
show_weighted_average('SE.PRM.ENRL.TC.ZS', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs', 'OECD'])

In [ ]:
# Population 0-14
show_weighted_average('SP.POP.0014.TO.ZS', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs', 'OECD'])

In [ ]:
years = list(range(1960,2018))
show_group_total('SP.POP.TOTL', ['LDCs'])

In [ ]:
data_group = get_data_table(years, 'SP.POP.TOTL', [['LDCs']])

In [ ]:
# Labor force
years = list(range(2000,2018))
show_weighted_average('SL.EMP.VULN.FE.ZS', 'SL.TLF.TOTL.IN', ['LDCs', 'Developing excluding LDCs'])

In [11]:
# Labor force
years = list(range(2011,2018))
show_weighted_average('SH.STA.MMRT', 'SP.POP.TOTL.FE.IN', ['LDCs', 'Developing excluding LDCs'])

TypeError: show_weighted_average() missing 1 required positional argument: 'groups'

In [14]:
import os.path
from os import path

In [20]:
from sources.worldbank.indicators import get
from countries.list import CountryList
from countries.utils import extract_countries
c = CountryList()
c.load_wb()
import json
import os.path
import time

In [21]:
data = get(name='SL.EMP.VULN.FE.ZS', 
           years=years, 
           countries=[c[100], c[101]])

In [22]:
n = 'data/SL.EMP.VULN.FE.ZS' + "-".join(list(map(str,years))) + ".json"
n2 = 'data/SL.EMP.VULN.FE.ZS2' + ".json"
with open(n, 'w') as outfile:
    json.dump(data, outfile)
    
outfile.close()

In [23]:
with open(n, 'r') as f:
    data_due = json.load(f)

In [24]:
os.path.getmtime(n) - time.time()

-1.1722474098205566

In [25]:
data_due

{'name': 'SLEMPVULNFEZS',
 'description': 'Vulnerable employment, female (% of female employment) (modeled ILO estimate)',
 'source': 'World Bank',
 'sourceurl': 'http://data.worldbank.org/indicator/SL.EMP.VULN.FE.ZS',
 'years': [2001,
  2002,
  2003,
  2004,
  2005,
  2006,
  2007,
  2008,
  2009,
  2010,
  2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017],
 'data': [{'entity': 'France', 'date': '2017', 'value': '6.28199988603592'},
  {'entity': 'France', 'date': '2016', 'value': '6.12100011110306'},
  {'entity': 'France', 'date': '2015', 'value': '5.97300010919571'},
  {'entity': 'France', 'date': '2014', 'value': '6.09400022029877'},
  {'entity': 'France', 'date': '2013', 'value': '5.52799981832505'},
  {'entity': 'France', 'date': '2012', 'value': '5.41300004720688'},
  {'entity': 'France', 'date': '2011', 'value': '5.48200011253357'},
  {'entity': 'France', 'date': '2010', 'value': '5.47099977731705'},
  {'entity': 'France', 'date': '2009', 'value': '5.11600005626678'},
  {'en

1571539930.448769